<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [12]:
#Start Here
import pandas as pd
df = pd.read_csv('./data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

In [13]:
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [14]:
'''Base stuff'''
import os
import re

import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import spacy

from collections import Counter

''' Gensim Stuff '''
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import SnowballStemmer

In [15]:
''' Cleaning text and preparing for NLP '''
STOPWORDS = set(STOPWORDS).union(set(['the', 'aa', 'aaa', 'aaaaaa' ]))

def tokenize(text): 
    return [token for token in simple_preprocess(text) if token not in STOPWORDS and len(token) >=3]

''' Throwing in some regex cleaning just for good measure '''
def clean(text):
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = text.replace('\n', '')
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [16]:
df['tokens'] = df['reviews.text'].apply(tokenize, clean)
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,tokens
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht...","[order, item, bad, quality, missing, backup, s..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht...","[bulk, expensive, way, products, like]"
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht...","[duracell, price, happy]"
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht...","[work, brand, batteries, better, price]"
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht...","[batteries, long, lasting, price, great]"


In [19]:
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

In [20]:
# a dictionary representation of all the words in our corpus
id2word = corpora.Dictionary(df['tokens'])

In [21]:
# let's do some filtering by removing extreme values from the dataset
id2word.filter_extremes(no_below=.15, no_above=.85)

In [22]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [23]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=2
                  )

In [24]:
''' Printing out topics cleaned up Version '''

words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:5]) for t in words]
for id, t in enumerate(topics): 
    print(f" ------ Topic {id} ------")
    print(t, end="\n\n")

 ------ Topic 0 ------
good price batteries quality far

 ------ Topic 1 ------
screen tablet play amazon google

 ------ Topic 2 ------
cheap like present know things

 ------ Topic 3 ------
tablet good amazon nice use

 ------ Topic 4 ------
batteries long work battery brand

 ------ Topic 5 ------
kindle read easy books use

 ------ Topic 6 ------
apps amazon music version use

 ------ Topic 7 ------
great price works product value

 ------ Topic 8 ------
bought account use reviews new

 ------ Topic 9 ------
ipad screen memory cover content

 ------ Topic 10 ------
love battery life great perfect

 ------ Topic 11 ------
product excellent features loved good

 ------ Topic 12 ------
loves gift bought christmas grandson

 ------ Topic 13 ------
tablet easy old use year

 ------ Topic 14 ------
daughter amazon buy wifi free



In [25]:
''' This is very useful for finding words that arent useful to the overall analysis '''
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)
# for the most part, the topic "bubbles" are distinct. Except for 3 areas, topic bubbles are not overlapping, thus we
# we could reduce the number of topics by 5 to obtain greater separation among bubbles. 
# there are some stopwords that could be removed as well.

//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.133286  0.169877       1        1  15.281007
13     0.198338 -0.174982       2        1  11.960967
5      0.160646  0.000548       3        1   8.635725
3     -0.004853 -0.005708       4        1   7.051285
0     -0.285110  0.041028       5        1   6.383758
1      0.132695 -0.039460       6        1   6.364585
9      0.081877  0.045628       7        1   5.902885
7     -0.264203 -0.190664       8        1   5.857282
6      0.075188  0.160527       9        1   5.732460
14     0.033718  0.094845      10        1   5.389363
8      0.083072  0.148124      11        1   4.943473
12     0.169682 -0.175597      12        1   4.826257
10    -0.061672 -0.127470      13        1   4.436835
11    -0.168406 -0.101911      14        1   3.963135
2     -0.017685  0.155214      15        1   3.270988, topic_info=     Category         Freq        Term        Total  loglift  logprob
22    Default  8725.000000       great  8725.000000  30.0000  30.0000
35    Default  5743.000000        good  5743.000000  29.0000  29.0000
4128  Default  6684.000000      tablet  6684.000000  28.0000  28.0000
19    Default  6962.000000   batteries  6962.000000  27.0000  27.0000
18    Default  4721.000000       price  4721.000000  26.0000  26.0000
3100  Default  2956.000000      kindle  2956.000000  25.0000  25.0000
90    Default  3439.000000        love  3439.000000  24.0000  24.0000
169   Default  2037.000000     product  2037.000000  23.0000  23.0000
2016  Default  2103.000000        apps  2103.000000  22.0000  22.0000
2567  Default  2207.000000       loves  2207.000000  21.0000  21.0000
3     Default  2060.000000     battery  2060.000000  20.0000  20.0000
883   Default  2788.000000        easy  2788.000000  19.0000  19.0000
2205  Default  1578.000000      screen  1578.000000  18.0000  18.0000
587   Default  2050.000000         old  2050.000000  17.0000  17.0000
1039  Default  1090.000000        gift  1090.000000  16.0000  16.0000
104   Default  1675.000000        year  1675.000000  15.0000  15.0000
1583  Default  1104.000000    daughter  1104.000000  14.0000  14.0000
24    Default  2358.000000        long  2358.000000  13.0000  13.0000
96    Default  1507.000000       works  1507.000000  12.0000  12.0000
29    Default  3177.000000      bought  3177.000000  11.0000  11.0000
195   Default  2461.000000        kids  2461.000000  10.0000  10.0000
754   Default  1664.000000       games  1664.000000   9.0000   9.0000
2617  Default   832.000000        ipad   832.000000   8.0000   8.0000
59    Default   980.000000        life   980.000000   7.0000   7.0000
10    Default  2221.000000        work  2221.000000   6.0000   6.0000
32    Default   894.000000   christmas   894.000000   5.0000   5.0000
50    Default  1122.000000     perfect  1122.000000   4.0000   4.0000
3116  Default  1470.000000       books  1470.000000   3.0000   3.0000
916   Default  1189.000000        read  1189.000000   2.0000   2.0000
257   Default   673.000000   excellent   673.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
47    Topic15   338.795288       cheap   555.889526   2.9249  -3.4107
571   Topic15   117.059395       spend   175.698471   3.0140  -4.4734
1841  Topic15    83.032219        huge   128.776840   2.9812  -4.8169
2     Topic15   154.372025         bad   281.323120   2.8199  -4.1967
114   Topic15   219.041122        know   452.010925   2.6956  -3.8468
441   Topic15    56.123333        aren    83.965813   3.0172  -5.2085
603   Topic15    37.337250       bunch    51.736713   3.0939  -5.6161
106   Topic15   167.315536      things   536.122803   2.2556  -4.1162
2110  Topic15    55.249104  understand   105.155594   2.7765  -5.2242
398   Topic15   142.573929       right   488.351379   2.1889  -4.2762
556   Topic15    45.251068        pick    80.867538   2.8395  -5.4239
567   Topic15    41.339966   

In [37]:
lda.print_topics()
# From the lda topics, we could potentially reduce all topics to 2: batteries and tablets(including kindles/ipads). Tablets 
# or synonyms for tablets are mentioned in several topics and this should be reduced.

[(0,
  '0.212*"good" + 0.080*"price" + 0.050*"batteries" + 0.032*"quality" + 0.023*"far" + 0.022*"buy" + 0.021*"brands" + 0.020*"amazon" + 0.019*"value" + 0.017*"long"'),
 (1,
  '0.056*"screen" + 0.037*"tablet" + 0.027*"play" + 0.019*"amazon" + 0.019*"google" + 0.016*"games" + 0.016*"reading" + 0.015*"movies" + 0.015*"web" + 0.014*"books"'),
 (2,
  '0.033*"cheap" + 0.029*"like" + 0.023*"present" + 0.021*"know" + 0.016*"things" + 0.015*"thanks" + 0.015*"bad" + 0.014*"right" + 0.012*"lot" + 0.012*"big"'),
 (3,
  '0.173*"tablet" + 0.042*"good" + 0.029*"amazon" + 0.020*"nice" + 0.015*"use" + 0.013*"little" + 0.013*"price" + 0.013*"tablets" + 0.012*"prime" + 0.012*"slow"'),
 (4,
  '0.124*"batteries" + 0.032*"long" + 0.030*"work" + 0.020*"battery" + 0.018*"brand" + 0.016*"amazon" + 0.014*"time" + 0.013*"use" + 0.012*"duracell" + 0.011*"buy"'),
 (5,
  '0.099*"kindle" + 0.035*"read" + 0.030*"easy" + 0.030*"books" + 0.028*"use" + 0.028*"love" + 0.025*"reading" + 0.020*"light" + 0.016*"reader" +

In [38]:
id2word = corpora.Dictionary(df['tokens'])
id2word.filter_extremes(no_below=.15, no_above=.85)
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [39]:
# Based on the analysis above, I'm updating some parameters, such as number of topics
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=42,
                   num_topics = 2,
                   passes=20,
                   workers=6
                  )

//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * log

In [40]:
lda.print_topics()

[(0,
  '0.037*"tablet" + 0.023*"great" + 0.019*"use" + 0.017*"love" + 0.016*"easy" + 0.016*"kindle" + 0.014*"bought" + 0.013*"kids" + 0.012*"loves" + 0.012*"old"'),
 (1,
  '0.048*"batteries" + 0.041*"great" + 0.039*"good" + 0.038*"price" + 0.017*"long" + 0.017*"work" + 0.016*"battery" + 0.016*"amazon" + 0.014*"buy" + 0.012*"product"')]

In [41]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:5]) for t in words]
for id, t in enumerate(topics): 
    print(f" ------ Topic {id} ------")
    print(t, end="\n\n")

 ------ Topic 0 ------
tablet great use love easy

 ------ Topic 1 ------
batteries great good price long



## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling

In [ ]:
topic_coherence = pd.DataFrame.from_records(coherence_values)

In [ ]:
topic_coherence.head()